<a href="https://colab.research.google.com/github/belalabouzaid/siads699_team13_collab/blob/main/Notebooks/01_Data_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Objective
This notebook downloads the Landsat-8 images from Google Earth Engine after defining the boundaries of the region of interest. The images are then used to extract the relevant bands information which will be used as features for drought prediction. Finally, the features are then saved along with the target variable in a dataframe and exported to .csv format to be used in subsequent notebooks for analysis and model training/testing.

# Setting up environment

In [ ]:
!pip install wxee

In [ ]:
# Import libraries
import ee
import geemap
import os
import pandas as pd
import wxee
import rioxarray
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce

## Authenticating Google Earth Engine API

In [ ]:
# Authenticate Earth Engine API
ee.Authenticate()

In [ ]:
# Intialize Earth Engine API
ee.Initialize()

# Obtain Data
Setting up functions

In [ ]:
# https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2

def apply_scale_factors(image):
  """apply scaling factors per landsat guidelines"""
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [ ]:
def calculate_var(img):
    """compute and add multiple bands to an ee image collection"""
    ndvi = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    ndmi = img.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDMI')
    mndwi = img.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')
    evi = ee.Image().expression('2.5*((NIR-Red)/(NIR+6*Red-7.5*Blue+1))', {'NIR': img.select('SR_B5'), 'Red': img.select('SR_B4'), 'Blue': img.select('SR_B2')}).rename('EVI')
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {'NIR': img.select('SR_B5'), 'Red': img.select('SR_B4')}).rename('SAVI')
    msavi = ee.Image().expression('(2*NIR+1-sqrt(pow((2*NIR+1),2)-8*(NIR-Red)))/2', {'NIR': img.select('SR_B5'), 'Red': img.select('SR_B4')}).rename('MSAVI')
    msi = img.select('SR_B6').divide(img.select('SR_B5')).rename('MSI')

    new_bands = ee.Image([ndvi, ndmi, mndwi, evi, savi, msavi, msi])
    return img.addBands(new_bands)

In [ ]:
def save_landsat8(roi, date_range, bands, roiname, datename):
    """function to fetch the various bands using ee and save the geotiffs, returning the metadata for the bands"""
    # Select the satellite image collection
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

    # Filter the image collection by date and region of interest(roi)
    image_collection = image_collection.filterDate(date_range).filterBounds(roi)

    # Scale the data per USGS guidelines
    image_collection = image_collection.map(apply_scale_factors)

    # Calculate additional variables and add the bands to the collection
    image_collection = image_collection.map(calculate_var)
    image_collection = image_collection.select(bands)

    # sort so percent cloud cover is smallest to largest select the most cloud free image in our filtered set
    image_collection = image_collection.sort("CLOUD_COVER").first()

    # save
    #after scaling file is too large so run through loop to sep. bands
    for band in bands:
        geemap.ee_export_image(image_collection.select(band),filename = str(band)+roiname+datename+'_ls8.tif',region = roi)

    #show metadata
    return geemap.image_props(image_collection)

In [ ]:
# some inspiration # https://docs.dea.ga.gov.au/notebooks/How_to_guides/Opening_GeoTIFFs_NetCDFs.html

def geotiff_df(files, roiname):
    """convert geotiffs to xarrays and create and return one dataframe of all data"""
    # create an empty list to append dataframes to
    dfs = []
    # read through files
    for f in files:
        # get variable name
        var = f.partition(roiname)[0]
        # open into xarray.DataArray
        da = rioxarray.open_rasterio(f)
        #convert to xarray.Dataset
        ds = da.to_dataset('band').rename({1: var})
        #convert to dataframe
        df = ds.to_dataframe().drop(columns=['spatial_ref'])
        # append df to list of dataframes
        dfs.append(df)
        # # no longer need file, so delete to save space
        # del f

    return reduce(lambda  left,right: pd.merge(left,right, on=['x','y']), dfs)

## Three Rivers, CA

In [ ]:
# area is 14.06 sqMiles or 36 sqKilometers
ThreeRivers_polygon = [[
                         [-118.89959054,36.48577439]
                        ,[-118.8270702,36.4862047]
                        ,[-118.82660683,36.43570829]
                        ,[-118.89912718,36.43527771]
                        ]]

### Obtain Spectral Data

Need spectral data for the first part of the drought (and as inputs to the model). This time frame is typically early in the season.

In [ ]:
input_bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','ST_B10','NDVI','NDMI','MNDWI','EVI','SAVI','MSAVI','MSI']

#### Testing and Training Data

In [ ]:
# Inputs to the function to get testing and training data for Three Rivers
# 2022 was the driest year so this will be best to use for testing and training

save_landsat8(
    roi = ee.Geometry.Polygon(ThreeRivers_polygon),
    date_range = ee.DateRange('2022-05-01', '2022-05-31'),
    bands = input_bands,
    roiname = 'ThreeRivers',
    datename = 'May2022')

Landsat Imagery for Three Rivers Training and Testing is from 2022-05-13

#### Validation Data

In [ ]:
# Inputs to the function to get validation data
# 2021 was another dry year so this will be good to use for validation

save_landsat8(
    roi = ee.Geometry.Polygon(ThreeRivers_polygon),
    date_range = ee.DateRange('2021-06-01', '2021-06-30'),
    bands = input_bands,
    roiname = 'ThreeRivers',
    datename = 'June2021')

Landsat Imagery for Three Rivers Validation is from 2021-06-11

### Create Dataframe of Data

In [ ]:
# inputs to function and run the function to create a dataframe for ThreeRivers 2022
# these are for training and testing

roiname='ThreeRivers'
files = [f for f in os.listdir() if f.endswith('ThreeRiversMay2022_ls8.tif')]
ThreeRivers_2022_df = geotiff_df(files, roiname)

In [ ]:
# inputs to function and run the function to create a dataframe for ThreeRivers 2021
# these are for validation

roiname='ThreeRivers'
files = [f for f in os.listdir() if f.endswith('ThreeRiversJune2021_ls8.tif')]
ThreeRivers_2021_df = geotiff_df(files, roiname)

### Elevation Data

Using Esri's ArcGIS Pro, imported an example geotiff (one that was retrieved above) to Esri for the area of interest. Terrain derivatives created from Digital Terrain Model (DTM) are available in Esri. Added slope and aspect layers (description of data: https://www.arcgis.com/home/item.html?id=58a541efc59545e6b7137f961d7de883) to the map and exported the raster data using the imported geotiff for the extent, the coordinate system, and the raster properties (looked at the Source properties, raster information to get number of columns and rows or alternately cell size) and maintained clipping extent. Then, still within Esri, created a csv of the upscaled data to import here.

In [ ]:
# Terrain: slope and aspect
# import csv but without first column of OIDs (ESRI byproduct)
# https://www.statology.org/pandas-read-csv-ignore-first-column/
# slope values are in degrees and aspect values are in cardinal direction with 361 as flat ground.

# ThreeRivers area
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/threerivers_dem_var.csv') as x:
    ncols = len(x.readline().split(','))
threerivers_dem_var = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/threerivers_dem_var.csv', usecols=range(1,ncols))

In [ ]:
# merge data dataframes with appropriate dem variables df

#2021 data
ThreeRivers_June2021_df = pd.merge(threerivers_dem_var, ThreeRivers_2021_df, how='left', left_on=['X','Y'], right_on = ['x','y'])
ThreeRivers_June2021_df.rename(columns={'aspect_Band_1': 'aspect', 'slope_Band_1': 'slope'}, inplace=True)

#2022 data
ThreeRivers_May2022_df = pd.merge(threerivers_dem_var, ThreeRivers_2022_df, how='left', left_on=['X','Y'], right_on = ['x','y'])
ThreeRivers_May2022_df.rename(columns={'aspect_Band_1': 'aspect', 'slope_Band_1': 'slope'}, inplace=True)

### SMI Data

The target variable for the last part of the drought, the Soil Moisture Index, was calculated in ArcGIS Pro and brought into this workspace.  The data for the last part of the drought is typically several months later, so usually in June, July, or August.
Using Esri's ArcGIS Pro, imported geotiffs of a cloud free image of Bands 4, 5, 10, 11 from Landsat OLI/TIRS C2 L1 from https://earthexplorer.usgs.gov for the region of interest. Performed raster calculations based on https://www.youtube.com/watch?v=KeIYja7CaFg of the method that is referenced by our inspiration study in Korea and detailed in the paper here: https://ieeexplore.ieee.org/document/1370089. SMI was calculated accordingly and brought into Python to join with input data from March.

In [ ]:
# import csv but without first column of OIDs (ESRI byproduct)
# https://www.statology.org/pandas-read-csv-ignore-first-column/
# soil moisture values range from 0 (dry) to 1 (wet)

# #2022 - Training and Testing - Data from July 16, 2022
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_ThreeRivers_2022.csv') as x:
    ncols = len(x.readline().split(','))
smi_ThreeRivers_2022 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_ThreeRivers_2022.csv', usecols=range(1,ncols))
smi_ThreeRivers_2022.rename(columns = {'smi22_Clip_Band_1': 'smi'}, inplace=True)

# #2021 - Validation - Data from August 30, 2021
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_ThreeRivers_2021.csv') as x:
    ncols = len(x.readline().split(','))
smi_ThreeRivers_2021 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_ThreeRivers_2021.csv', usecols=range(1,ncols))
smi_ThreeRivers_2021.rename(columns = {'smi21_Clip_Band_1': 'smi'}, inplace=True)

In [ ]:
ThreeRivers_2022 = pd.merge(ThreeRivers_May2022_df, smi_ThreeRivers_2022)
ThreeRivers_2021 = pd.merge(ThreeRivers_June2021_df, smi_ThreeRivers_2021)

### Export Finalized Data to CSV

In [ ]:
# ThreeRivers_2022.to_csv('/content/data/threerivers_2022.csv', index=False)
# ThreeRivers_2021.to_csv('/content/data/threerivers_2021.csv', index=False)

## Mariposa, CA

In [ ]:
# area is 20.69 sqMiles or 54 sqKilometers
mariposa_polygon = [[
                         [-119.93332186358441,37.492038950610564]
                        ,[-120.02430507710635,37.491733062301975]
                        ,[-120.02462577206953,37.55175969111435]
                        ,[-119.93364255854758,37.55206533342423]
                        ]]

### Obtain Spectral Data

In [ ]:
input_bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','ST_B10','NDVI','NDMI','MNDWI','EVI','SAVI','MSAVI','MSI']

#### Testing and Training Data

In [ ]:
# Inputs to the function to get the testing and training data
# 2021 was the driest year for Mariposa so this will be best to use for testing and training

save_landsat8(
    roi = ee.Geometry.Polygon(mariposa_polygon),
    date_range = ee.DateRange('2021-05-01', '2021-05-31'),
    bands = input_bands,
    roiname = 'mariposa',
    datename = 'May2021')

Landsat Imagery for Mariposa Training and Testing is from 2021-05-01

#### Validation Data

In [ ]:
# Inputs to the function to get validation data
# 2022 was another dry year so this will be good to use for validation

save_landsat8(
    roi = ee.Geometry.Polygon(mariposa_polygon),
    date_range = ee.DateRange('2022-05-01', '2022-05-31'),
    bands = input_bands,
    roiname = 'mariposa',
    datename = 'May2022')

Landsat Imagery for Mariposa Training and Testing is from 2022-05-20

### Create Dataframe of Data

In [ ]:
# inputs to function and run the function to create a dataframe for Mariposa 2021
# this is for training and testing

roiname='mariposa'
files = [f for f in os.listdir() if f.endswith('mariposaMay2021_ls8.tif')]
mariposa_2021_df = geotiff_df(files, roiname)

In [ ]:
# inputs to function and run the function to create a dataframe for Mariposa 2022
# this is for validation

roiname='mariposa'
files = [f for f in os.listdir() if f.endswith('mariposaMay2022_ls8.tif')]
mariposa_2022_df = geotiff_df(files, roiname)

### Elevation Data

In [ ]:
# Terrain: slope and aspect
# import csv but without first column of OIDs (ESRI byproduct)
# https://www.statology.org/pandas-read-csv-ignore-first-column/
# slope values are in degrees and aspect values are in cardinal direction with 361 as flat ground.

# Mariposa area
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/mariposa_dem_var.csv') as x:
    ncols = len(x.readline().split(','))
mariposa_dem_var = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/mariposa_dem_var.csv', usecols=range(1,ncols))

In [ ]:
# merge data dataframes with appropriate dem variables df

#2021 data
mariposa_May2021_df = pd.merge(mariposa_dem_var, mariposa_2021_df, how='left', left_on=['X','Y'], right_on = ['x','y'])
mariposa_May2021_df.rename(columns={'aspect_Band_1': 'aspect', 'slope_Band_1': 'slope'}, inplace=True)

#2022 data
mariposa_May2022_df = pd.merge(mariposa_dem_var, mariposa_2022_df, how='left', left_on=['X','Y'], right_on = ['x','y'])
mariposa_May2022_df.rename(columns={'aspect_Band_1': 'aspect', 'slope_Band_1': 'slope'}, inplace=True)

### SMI Data

In [ ]:
# import csv but without first column of OIDs (ESRI byproduct)
# https://www.statology.org/pandas-read-csv-ignore-first-column/
# soil moisture values range from 0 (dry) to 1 (wet)

# 2021 - Training and Testing - Data from July 20, 2021
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_mariposa_2021.csv') as x:
    ncols = len(x.readline().split(','))
smi_mariposa_2021 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_mariposa_2021.csv', usecols=range(1,ncols))
smi_mariposa_2021.rename(columns = {'smi_21_Band_1': 'smi'}, inplace=True)

# #2022 - Validation - Data from July 23, 2022
with open('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_mariposa_2022.csv') as x:
    ncols = len(x.readline().split(','))
smi_mariposa_2022 = pd.read_csv('https://github.com/belalabouzaid/siads699_team13_collab/blob/main/Data/smi_mariposa_2022.csv', usecols=range(1,ncols))
smi_mariposa_2022.rename(columns = {'smi_22_Band_1': 'smi'}, inplace=True)

In [ ]:
#merge smi with data
mariposa_2021 = pd.merge(mariposa_May2021_df, smi_mariposa_2021)
mariposa_2022 = pd.merge(mariposa_May2022_df, smi_mariposa_2022)

### Export Finalized Data to CSV

In [ ]:
# mariposa_2021.to_csv('/content/data/mariposa_2021.csv', index=False)
# mariposa_2022.to_csv('/content/data/mariposa_2022.csv', index=False)